In [1]:
import requests
from classes.card import Card

# Card data from the-fab-cube
url = "https://raw.githubusercontent.com/the-fab-cube/flesh-and-blood-cards/develop/json/english/card.json"

card_data = requests.get(url).json()

all_cards = []
for card in card_data:
    card_obj = Card(card)
    all_cards.append(card_obj)

In [2]:
from itertools import combinations

def generate_card_combinations(cards, n):
    """
    Generate all unique combinations of size n from the given list of cards.

    Args:
        cards (List[Card]): List of Card objects.
        n (int): The size of each combination.

    Returns:
        List[Tuple[Card, ...]]: A list of tuples, each containing a combination of cards.
    """
    return list(set(combinations(cards, n)))

# Fai

In [6]:
fai_deck_string = """
Deck built with ❤️ at the FaBrary

World Championship: Barcelona 1st 🇬🇷 Copy

Class: Ninja
Hero: Fai, Rising Rebellion
Weapons: Harmonized Kodachi,Harmonized Kodachi, Searing Emberblade
Equipment: Flamescale Furnace, Fyendal's Spring Tunic, Mask of Momentum, Mask of the Pouncing Lynx, Snapdragon Scalers, Tide Flippers, Tiger Stripe Shuko

(3) Ancestral Empowerment (red)
(3) Bittering Thorns (red)
(3) Blaze Headlong (red)
(3) Brand with Cinderclaw (red)
(3) Double Strike (red)
(3) Enlightened Strike (red)
(3) Lava Burst (red)
(3) Mounting Anger (red)
(3) Ravenous Rabble (red)
(3) Rising Resentment (red)
(3) Ronin Renegade (red)
(3) Snatch (red)
(3) Art of War (yellow)
(3) Brand with Cinderclaw (yellow)
(3) Salt the Wound (yellow)
(3) Brand with Cinderclaw (blue)
(3) Lava Vein Loyalty (blue)
(3) Soulbead Strike (blue)
(3) Stab Wound (blue)
(2) Warmonger's Diplomacy (blue)


See the full deck @ https://fabrary.net/decks/01HH37B8K3ZDXBGCT4R7YDGDFG
"""

In [7]:
from classes.deck import Deck

fai_deck = Deck(fai_deck_string, all_cards)
fai_full_handset = fai_deck.calculate_hands()
fai_full_handset = fai_full_handset.filter("arsenal_is", None).filter("has_n_cards", 4)

Calculating permutations for hand size 0


1it [00:00, ?it/s]


Calculating permutations for hand size 1


59it [00:00, ?it/s]


Calculating permutations for hand size 2


1711it [00:00, 32903.06it/s]


Calculating permutations for hand size 3


32509it [00:00, 38976.45it/s]


Calculating permutations for hand size 4


455126it [00:11, 38212.99it/s]


## Tests of aggregate

In [4]:
conditions = {}

# Generate all blues
blue_card = fai_deck.group_by(lambda card: card.pitch == 3)
conditions["One Blue"] = generate_card_combinations(blue_card, 1)
conditions["Two Blues"] = generate_card_combinations(blue_card, 2)
conditions["Three Blues"] = generate_card_combinations(blue_card, 3)


# Generate draconic chain starter non-blue
non_blue_draconic_chain_starter =  fai_deck.group_by(lambda card: ("Attack" in card.types
                                                          and "Draconic" in card.types
                                                          and card.cost == 0
                                                          and "Go again" in card.functional_text_plain
                                                          and card.pitch != 3
                                                          )
                                            )
conditions["A 0-cost, non-Blue, draconic chain opener"] = generate_card_combinations(non_blue_draconic_chain_starter, 1)
conditions["Two 0-cost, non-Blue, draconic chain openers"] = generate_card_combinations(non_blue_draconic_chain_starter, 2)

non_blue_natural_go_again =  fai_deck.group_by(lambda card: ("Attack" in card.types
                                                          and "Go again" in card.functional_text_plain
                                                          and card.pitch != 3
                                                          )
                                            )
conditions["A non-blue natural go again attack"] = generate_card_combinations(non_blue_natural_go_again, 1)
conditions["Two non-blue natural go again attacks"] = generate_card_combinations(non_blue_natural_go_again, 2)

non_blue_one_cost_go_again =  fai_deck.group_by(lambda card: (card.name == "Mounting Anger"
                                                              or card.name == "Bittering Thorns")
                                            )
conditions[""] = generate_card_combinations(non_blue_natural_go_again, 2)

# Generate Lava Burst
lava_burst = fai_deck.group_by(lambda card: card.name == "Lava Burst")
a_lava_burst = generate_card_combinations(lava_burst, 1)

#Generate block 3+
any_card = fai_deck.cards
any_card_at_all = generate_card_combinations(any_card, 1)


# Specify condition test names
conditions = {"Blue": a_blue,
              "Chain Starter": a_chain_starter,
              "Lava Burst": a_lava_burst,
              "Any Card": any_card_at_all}

fai_full_handset.analyze_card_list_combinations(conditions)

,Blue,Chain Starter,Lava Burst,Any Card,Hand Counts,Combinations,Percentage
0,True,True,True,True,330,21672,3.74%
1,True,True,True,False,0,0,0.00%
2,True,True,False,True,2586,147014,29.33%
3,True,True,False,False,0,0,0.00%
4,True,False,True,True,529,29099,6.00%
5,True,False,True,False,0,0,0.00%
6,True,False,False,True,2326,108346,26.38%
7,True,False,False,False,0,0,0.00%
8,False,True,True,True,394,23550,4.47%
9,False,True,True,False,0,0,0.00%


## Rest of code

In [5]:
from functions.generatehandset import generate_handsets

lava_burst = fai_deck.group_by(lambda card: card.name == "Lava Burst")
block_3_or_more = fai_deck.group_by(lambda card: card.defense >= 3)
# any_card = fai_deck.cards
blue_card = fai_deck.group_by(lambda card: card.pitch == 3)
non_blue_draconic_chain_starter =  fai_deck.group_by(lambda card: ("Attack" in card.types
                                                          and "Draconic" in card.types
                                                          and card.cost == 0
                                                          and "Go again" in card.functional_text_plain
                                                          and card.pitch != 3
                                                          )
                                            )

arsenal_options = None

card_options_tuple = (lava_burst, block_3_or_more, blue_card, non_blue_draconic_chain_starter)
lava_burst_handset_3_block = generate_handsets(fai_deck, card_options_tuple, arsenal_options)
lava_burst_handset_3_block

Handset object with 136 distinct hands, and 8208 hand combinations.

In [6]:
any_card = fai_deck.cards
card_options_tuple = (any_card, any_card, any_card, any_card)
generate_handsets(fai_deck, card_options_tuple, None)

Handset object with 8816 distinct hands, and 455126 hand combinations.

In [7]:
lava_burst = fai_deck.group_by(lambda card: card.name == "Lava Burst")
block_3_or_more = fai_deck.group_by(lambda card: card.defense >= 3)
any_card = fai_deck.cards
blue_card = fai_deck.group_by(lambda card: card.pitch == 3)
non_blue_draconic_chain_starter =  fai_deck.group_by(lambda card: ("Attack" in card.types
                                                          and "Draconic" in card.types
                                                          and card.cost == 0
                                                          and "Go again" in card.functional_text_plain
                                                          and card.pitch != 3
                                                          )
                                            )

arsenal_options = None

card_options_tuple = (lava_burst, any_card, blue_card, non_blue_draconic_chain_starter)
lava_burst_handset_any_4th = generate_handsets(fai_deck, card_options_tuple, arsenal_options)
lava_burst_handset_any_4th

Handset object with 330 distinct hands, and 21672 hand combinations.

In [8]:
lava_burst = fai_deck.group_by(lambda card: card.name == "Lava Burst")
any_card = fai_deck.cards

arsenal_options = None

card_options_tuple = (lava_burst, any_card, any_card, any_card)
lava_burst_handset_any_cards = generate_handsets(fai_deck, card_options_tuple, arsenal_options)
lava_burst_handset_any_cards

Handset object with 1538 distinct hands, and 87836 hand combinations.

In [9]:
fai_4_card_handset_no_arse = fai_full_handset.filter("has_n_cards", 4).filter("arsenal_is", None)


In [10]:
lava_burst_handset_any_4th.difference(lava_burst_handset_3_block)

Handset object with 194 distinct hands, and 13464 hand combinations.

## Aggregate tests

In [11]:
aggregated_data = lava_burst_handset_any_4th.aggregate([
    lambda hand: any(card in blue_card for card in hand.hand),
    lambda hand: any(card in non_blue_draconic_chain_starter for card in hand.hand)
])
aggregated_data.keys()

dict_keys([0, 1])

# Bravo List Comparison

In [3]:
bravo_string_1 = """
Deck built with ❤️ at the FaBrary

$20k Invitational

Class: Guardian
Hero: Bravo, Showstopper
Weapons: Anothos, Titan's Fist
Equipment: Civic Steps, Crater Fist, Crown of Providence, Fyendal's Spring Tunic, Nullrune Boots, Steelbraid Buckler

(2) Chokeslam (red)
(3) Command and Conquer (red)
(3) Crippling Crush (red)
(3) Enlightened Strike (red)
(2) Pummel (red)
(3) Sink Below (red)
(3) Spinal Crush (red)
(3) Star Struck (yellow)
(2) Boulder Drop (blue)
(3) Buckling Blow (blue)
(3) Chokeslam (blue)
(3) Cranial Crush (blue)
(3) Debilitate (blue)
(3) Disable (blue)
(3) Imposing Visage (blue)
(3) Macho Grande (blue)
(3) Rouse the Ancients (blue)
(1) Show Time! (blue)
(3) Tear Asunder (blue)
(3) Thunder Quake (blue)
(2) Unmovable (blue)
(3) Warmonger's Diplomacy (blue)


See the full deck @ https://fabrary.net/decks/01HGVPG2C92B6K08317CCNM8KD
"""

bravo_string_2 = """
Deck built with ❤️ at the FaBrary

$20k Invitational

Class: Guardian
Hero: Bravo, Showstopper
Weapons: Anothos, Titan's Fist
Equipment: Civic Steps, Crater Fist, Crown of Providence, Fyendal's Spring Tunic, Nullrune Boots, Steelbraid Buckler

(2) Chokeslam (red)
(3) Command and Conquer (red)
(3) Crippling Crush (red)
(2) Enlightened Strike (red)
(2) Pummel (red)
(3) Sink Below (red)
(3) Spinal Crush (red)
(3) Star Struck (yellow)
(2) Boulder Drop (blue)
(3) Buckling Blow (blue)
(3) Chokeslam (blue)
(3) Cranial Crush (blue)
(3) Debilitate (blue)
(3) Disable (blue)
(3) Imposing Visage (blue)
(3) Macho Grande (blue)
(3) Rouse the Ancients (blue)
(2) Show Time! (blue)
(3) Tear Asunder (blue)
(3) Thunder Quake (blue)
(2) Unmovable (blue)
(3) Warmonger's Diplomacy (blue)


See the full deck @ https://fabrary.net/decks/01HGVPG2C92B6K08317CCNM8KD
"""

In [9]:
from classes.deck import Deck

bravo_deck_1 = Deck(bravo_string_2, all_cards)
bravo_1_handset = bravo_deck_1.calculate_hands()
bravo_1_handset = bravo_1_handset.filter("has_n_cards", 4).filter("arsenal_is", None)

# bravo_deck_2 = Deck(bravo_string_2, all_cards)
# bravo_2_handset = bravo_deck_2.calculate_hands()
# bravo_2_handset = bravo_2_handset.filter("has_n_cards", 4).filter("arsenal_is", None)

Calculating permutations for hand size 0


1it [00:00, ?it/s]


Calculating permutations for hand size 1


60it [00:00, 7104.57it/s]


Calculating permutations for hand size 2


1770it [00:00, 31189.66it/s]


Calculating permutations for hand size 3


34220it [00:00, 34393.90it/s]


Calculating permutations for hand size 4


487635it [00:13, 36862.09it/s]


In [10]:
conditions = {}

four_cost_atk = bravo_deck_1.group_by(lambda card: (card.pitch == 1 and (card.name == "Chokeslam" or card.name == "Spinal Crush")))
conditions["Chokey or Spinal"] = generate_card_combinations(four_cost_atk, 1)

seven_cost_atk = bravo_deck_1.group_by(lambda card: (card.name == "Crippling Crush" or card.name == "Star Struck"))
conditions["Crip or Star"] = generate_card_combinations(seven_cost_atk, 1)

sink = bravo_deck_1.group_by(lambda card: card.name == "Sink Below")
conditions["Sink Below"] = generate_card_combinations(sink, 1)

cnc_pummel = bravo_deck_1.group_by(lambda card: (card.name == "Command and Conquer"
                                            or card.name == "Pummel"
                                           )
                             )

cnc = bravo_deck_1.group_by(lambda card: card.name == "Command and Conquer")
pummel = bravo_deck_1.group_by(lambda card: card.name == "Pummel")
conditions["CnC"] = generate_card_combinations(cnc, 1)
conditions["Pummel"] = generate_card_combinations(pummel, 1)

estrike = bravo_deck_1.group_by(lambda card: card.name == "Enlightened Strike")
conditions["Estrike"] = generate_card_combinations(estrike, 1)

blue_cards = bravo_deck_1.group_by(lambda card: card.pitch == 3)

conditions["A Blue"] = generate_card_combinations(blue_cards, 1)
conditions["Two Blues"] = generate_card_combinations(blue_cards, 2)
conditions["Three Blues"] = generate_card_combinations(blue_cards, 3)
conditions["Four Blues"] = generate_card_combinations(blue_cards, 4)

analysis_df = bravo_1_handset.analyze_card_list_combinations(conditions)
analysis_df.to_csv("MH minus estrike Bravo Hand Analysis Matrix.csv")
analysis_df

0it [00:00, ?it/s]

c:\Python Projects\Jupyter\fun\fabsim\classes\handset.py:201: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_row], ignore_index=True)
1024it [02:07,  8.05it/s]


,Chokey or Spinal,Crip or Star,Sink Below,CnC,Pummel,Estrike,A Blue,Two Blues,Three Blues,Four Blues,Hand Counts,Combinations,Percentage,Hands
0,False,False,False,False,False,False,True,True,True,True,2327,82251,0.186130,"((Macho Grande (3), Show Time! (3), Imposing V..."
2,True,False,False,False,False,False,True,True,True,False,1114,45695,0.089106,"((Spinal Crush (1), Rouse the Ancients (3), De..."
1,False,True,False,False,False,False,True,True,True,False,1114,54834,0.089106,"((Warmonger's Diplomacy (3), Buckling Blow (3)..."
3,False,False,False,False,False,True,True,True,True,False,557,18278,0.044553,"((Thunder Quake (3), Enlightened Strike (1), R..."
4,False,False,True,False,False,False,True,True,True,False,557,27417,0.044553,"((Macho Grande (3), Buckling Blow (3), Sink Be..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,True,False,False,True,True,True,False,False,False,False,2,60,0.000160,"((Spinal Crush (1), Enlightened Strike (1), Co..."
105,True,False,True,True,True,False,False,False,False,False,2,90,0.000160,"((Sink Below (1), Chokeslam (1), Command and C..."
104,False,True,True,False,True,True,False,False,False,False,2,72,0.000160,"((Sink Below (1), Enlightened Strike (1), Crip..."
117,False,False,True,True,True,True,False,False,False,False,1,36,0.000080,"((Sink Below (1), Enlightened Strike (1), Comm..."


In [11]:
import pandas as pd

mh = pd.read_csv("MH Bravo Hand Analysis Matrix.csv")
mh_blue = pd.read_csv("MH minus estrike Bravo Hand Analysis Matrix.csv")

In [19]:
# Columns to compare
comparison_columns = mh.columns.tolist()
comparison_columns.remove('Unnamed: 0')
comparison_columns.remove('Hand Counts')
comparison_columns.remove('Combinations')
comparison_columns.remove('Percentage')
comparison_columns.remove('Hands')


# Merge the DataFrames on the comparison columns
merged_df = pd.merge(mh, mh_blue, on=comparison_columns, how="outer", suffixes=('_38', '_39'))

# Handle missing values (set them to 0 or other value if needed)
merged_df.fillna({'Percentage_df1': 0, 'Percentage_df2': 0}, inplace=True)

# Calculate the difference in 'Percentage'
merged_df['Percentage Diff'] = merged_df['Percentage_38'] - merged_df['Percentage_39']

# Display the result
df_comparison = merged_df[comparison_columns + ['Hand Counts_38', 'Hand Counts_39', 'Combinations_38', 'Combinations_39', 'Percentage_38', 'Percentage_39', 'Percentage Diff']].sort_values(by="Percentage Diff", ascending=False)
df_comparison.to_csv("Bravo Difference.csv")